# Part X — Advanced Topics  
## 47. Managing Large Codebases (Modularization, Boundaries, Rules, Feature Flags, Dependency Discipline)

This chapter is about what happens when your Django project becomes a real product:

- many apps
- many developers
- multiple teams (frontend, platform, data, ops)
- frequent releases
- long-lived code
- legacy behavior you must not break

The goal is to keep the codebase:
- **modular** (changes don’t cascade unexpectedly)
- **testable** (fast feedback)
- **consistent** (shared patterns)
- **safe** (authorization and tenant scoping don’t drift)
- **operable** (logs/metrics are consistent)

We’ll focus on practical, enforceable rules that real teams use.

---

## 47.0 Learning Outcomes

By the end, you should be able to:

1. Define app boundaries and “ownership” rules for a monolith.
2. Organize code for scale:
   - app modules: selectors/services/policies/api
   - internal packages (core) vs domain apps
3. Prevent dependency spaghetti:
   - forbid cross-imports except through explicit APIs
   - avoid circular dependencies
4. Introduce feature flags safely (release without deploying risky behavior).
5. Create architectural enforcement:
   - static checks (import rules)
   - tests that enforce boundaries
6. Manage settings and environment configurations cleanly.
7. Adopt conventions for migrations, tasks, and event/audit logging at scale.
8. Build “engineering docs” that keep teams aligned.

---

# 47.1 The Core Problem in Big Django Monoliths

Big Django monoliths don’t fail because Django can’t scale—they fail because:

- business logic is scattered across views/models/forms/tasks
- “quick fixes” bypass selectors/services/policies
- people copy/paste query logic and forget scoping
- permissions drift across HTML and API implementations
- “core” becomes a dumping ground
- changes become risky because no one knows what depends on what

The solution is:
- explicit boundaries + conventions + enforcement

---

# 47.2 App Boundaries (How to Define “Modules” in Django)

### 47.2.1 Domain apps vs platform apps
**Domain apps**: represent your business domains:
- articles
- tasks
- orgs
- billing
- inventory
- support

**Platform/shared apps**: provide common infrastructure:
- core (utilities, base classes)
- audit (audit logging)
- integrations (external APIs/webhooks)
- realtime (Channels consumers)
- notifications (email/push abstraction)

### 47.2.2 A clean monorepo structure (industry common)

```text
config/                  # settings, urls, asgi/wsgi, middleware, logging
core/                    # shared utilities and base patterns
audit/                   # audit log model + service
integrations/            # external API clients + webhooks
realtime/                # Channels consumers + routing
articles/
orgs/
tasks/
tests/                   # optional global tests + factories + helpers
docs/
```

**Rule:** domain apps should not import each other freely.
Instead, they should use:
- explicit service APIs
- signals/events
- or “integration layer” modules

---

# 47.3 “Public APIs” for Apps (The Boundary Pattern)

A clean pattern is to define each app’s external interface via a small module, e.g.:

- `tasks/api.py` (not DRF, but internal “service API”)
- or `tasks/public.py`

Example: `tasks/public.py`

```python
# tasks/public.py
from __future__ import annotations

from tasks.services import create_task_from_form, update_task_from_form
from tasks.selectors import task_qs_for_org
from tasks.policies import can_edit_task, can_export_tasks

__all__ = [
    "create_task_from_form",
    "update_task_from_form",
    "task_qs_for_org",
    "can_edit_task",
    "can_export_tasks",
]
```

Now other apps import from `tasks.public`, not `tasks.services` directly.

### Why this helps
- you can change internal structure without breaking imports across the codebase
- you can audit and review cross-app dependencies
- you reduce circular import risk

This is a very common large-monolith technique.

---

# 47.4 Dependency Rules (Enforceable Boundaries)

Define rules like:

1. `articles` can depend on:
   - core
   - audit
   - integrations
   - but not `tasks` directly (unless via public APIs)
2. `tasks` can depend on:
   - orgs
   - core
   - audit
3. `integrations` must not import domain apps (or must import through explicit adapters)
4. `core` should not depend on domain apps (core is “lowest level”)

### Why “core must not depend on domain apps”
If core depends on domain apps, it becomes impossible to reuse core and you create
circular dependencies.

Core should be foundational.

---

# 47.5 Enforce Boundaries with Import Rules (Industry Standard Tools)

Python doesn’t enforce module boundaries by default. Big codebases do.

Common options:
- `import-linter` (explicit import contract rules)
- `flake8-tidy-imports` or Ruff rules + conventions
- custom tests that inspect imports

We’ll implement a simple enforcement using `import-linter` (conceptually; you can
choose another tool).

### 47.5.1 Install import-linter
Add to dev deps:

```text
import-linter
```

Install:

```bash
python -m pip install -r requirements-dev.txt
```

### 47.5.2 Add `importlinter` config
Create `importlinter.ini`:

```ini
[importlinter]
root_package = config

[importlinter:contract:core_independent]
name = Core must not import domain apps
type = forbidden
source_modules =
    core
forbidden_modules =
    articles
    tasks
    orgs
    integrations
    audit
    realtime
```

Run:

```bash
lint-imports
```

### Add to CI
Add a CI step:

```bash
lint-imports
```

This keeps architecture from degrading quietly.

> If your root package is not `config`, adjust `root_package`. Some teams set it to
> the project root and list packages explicitly.

---

# 47.6 Feature Flags (Release Safety in Large Codebases)

Feature flags let you:
- merge code that is not yet “on”
- enable gradually
- rollback behavior without rollback deploy

Two types:
- **static flags** (settings/env var)
- **dynamic flags** (DB-backed, per-tenant, per-user)

Start with static flags (simplest and very useful).

### 47.6.1 Static feature flag pattern
In `config/settings/base.py`:

```python
import os

FEATURE_TASK_EXPORT_V2 = os.environ.get("FEATURE_TASK_EXPORT_V2", "false").lower() == "true"
```

In code:

```python
from django.conf import settings

if settings.FEATURE_TASK_EXPORT_V2:
    # new export job flow
else:
    # old export flow
```

### 47.6.2 Dynamic feature flags (advanced)
When you need:
- enable per org
- enable for internal testers

You create:
- `FeatureFlag` model (name, enabled)
- `FeatureFlagOverride` (org/user based)

But dynamic flags require:
- caching
- admin UI
- audit logs
- security (who can toggle)

Implement when you actually need per-tenant rollouts.

---

# 47.7 Refactoring Strategy in Large Codebases (How Pros Avoid Big Bang)

### 47.7.1 Strangler pattern
- build the new implementation alongside old
- route a small percent of traffic/users/orgs to new
- expand coverage
- remove old once proven

Feature flags enable this.

### 47.7.2 Expand/contract for schema changes
You already learned this for migrations; it’s part of large codebase discipline.

---

# 47.8 Code Organization Conventions That Scale

These conventions keep a large Django project predictable:

## 47.8.1 Views are thin
- parse/validate
- call selectors/services/policies
- render/return

## 47.8.2 Selectors: “All queries live here”
- name them clearly:
  - `task_qs_for_org`
  - `get_task_for_org_or_404`
- always include scoping and prefetching

## 47.8.3 Services: “All workflows live here”
- create/update actions
- transactions
- background job enqueue calls (`on_commit`)
- integration calls

## 47.8.4 Policies: “All auth rules live here”
- return structured decisions
- tested heavily

## 47.8.5 Tasks: Celery tasks are thin wrappers
- call service logic
- handle retries/timeouts
- keep idempotency markers

## 47.8.6 Signals: use sparingly
Signals can create hidden behavior. Prefer explicit service calls.
Use signals when:
- decoupling is essential
- behavior is truly cross-cutting
- you document and test it

---

# 47.9 Documentation That Scales (Engineering Docs, Not Marketing)

A healthy big codebase has docs for:

- architecture overview
- how to add a new endpoint (view → form → selector → service → policy)
- authorization matrix
- tenancy rules
- migrations playbook
- operational runbooks
- API versioning/deprecation policy

Minimum docs folder:

```text
docs/
  architecture.md
  authorization.md
  tenancy.md
  migrations.md
  ops/
  api.md
  development.md
```

---

# 47.10 Dependency Management at Scale

Large projects have dependency discipline:

- pinned runtime deps (requirements lock)
- dev deps separated
- periodic upgrade cadence (monthly)
- security patch fast lane
- avoid unnecessary dependencies

In Python/Django teams, common practice:
- dependabot/renovate for update PRs
- `pip-audit` in CI
- treat major upgrades as projects, not “one afternoon”

---

# 47.11 Testing Strategy for Large Codebases

As code grows, test speed matters.

Professional strategies:
- keep unit tests fast and plentiful
- isolate DB usage; use factories
- avoid heavy E2E tests on every PR
- run contract fuzzing nightly
- keep query-count tests for hot endpoints
- track test runtime in CI and optimize

---

# 47.12 Capstone Lab: Enforce Architecture Rules in Your Repo

1. Define app dependency rules:
   - core cannot import domain apps
   - integrations cannot import domain apps directly
   - tasks can import orgs; articles cannot import tasks
2. Add import-linter config.
3. Run lint-imports locally; fix violations.
4. Add lint-imports to CI.
5. Add docs describing:
   - app boundaries
   - allowed dependencies
   - where to put new code

---

## 47.13 Exercises (Do These Before Proceeding)

1. Create `docs/architecture.md` that includes:
   - a diagram of apps and allowed dependencies
   - your layers (views/forms/selectors/services/policies)
2. Add one feature flag and use it to:
   - toggle “export v2” flow on/off
   - write a test verifying both behaviors work
3. Refactor one cross-app import to use a public API module.
4. Add a “no signals without documentation” rule:
   - list signals in a `docs/signals.md`
5. Add a CI job that runs:
   - lint/format/test/import rules/migrations check

---

## 47.14 Chapter Summary

- Large Django codebases need explicit boundaries and enforcement.
- Define domain vs platform apps and restrict dependencies.
- Expose app “public APIs” to prevent import spaghetti.
- Use feature flags for safe rollout and strangler refactors.
- Keep code layered: selectors/services/policies make behavior consistent and testable.
- Architecture documentation and CI rules keep teams aligned over time.

---

Next chapter: **Capstone Projects (Part X — 48–50)**  
You’ll choose one capstone track and build a production-grade app end-to-end,
including deployment, monitoring, security review, and operational runbooks:

- Capstone A: Content platform (membership, protected content, editorial workflows)
- Capstone B: SaaS CRUD platform (multi-tenant + DRF + background jobs + CI/CD)
- Capstone C: Realtime app (Channels + scaling + observability)